In [173]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import torch
import random

from sae.sparse_autoencoder import load_saved_sae
from sae.metrics import model_store_from_sae
from unlearning.metrics import convert_wmdp_data_to_prompt, convert_list_of_dicts_to_dict_of_lists
from unlearning.tool import UnlearningConfig, SAEUnlearningTool, MCQ_ActivationStoreAnalysis, ActivationStoreAnalysis
from unlearning.metrics import modify_and_calculate_metrics, calculate_metrics_list, create_df_from_metrics

from huggingface_hub import hf_hub_download
from datasets import load_dataset
import numpy as np
import pandas as pd
import itertools
from transformer_lens import utils

from jaxtyping import Float
from torch import Tensor

from pathlib import Path

import plotly.express as px
from unlearning.var import REPO_ID, SAE_MAPPING
import pickle

from unlearning.metrics import all_permutations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Load main SAE for gemma-2b-it
filename = hf_hub_download(repo_id=REPO_ID, filename=SAE_MAPPING['gemma_2b_it_resid_pre_9'])
sae = load_saved_sae(filename)
model = model_store_from_sae(sae)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
Moving model to device:  cuda


In [3]:
activation_store = ActivationStoreAnalysis(sae.cfg, model)

buffer
dataloader


In [208]:
# load good feature list

with open('../yeutong_notebooks/unlearning_output/good_features_list_v1.pkl', 'rb') as f:
    good_features_list = pickle.load(f)


features_to_test = list(set([item for sublist in good_features_list.values() for item in sublist]))

filtered_good_features = [12663, 4342, 5749, 10355, 1523, 15858, 12273, 14315, 4451, 1611, 10051, 16186, 7983, 6958, 1307, 11019, 6531, 12289]
filtered_features_sorted_by_loss = [7983, 16186, 12273, 14315,  4342, 10051, 15858,  6958, 12663, 1611,  6531,  1523, 10355,  5749,  1307, 12289,  4451, 11019]
filtered_features_sorted_by_loss2 = np.concatenate((filtered_features_sorted_by_loss[:8], filtered_features_sorted_by_loss[10:11], filtered_features_sorted_by_loss[12:]))

zero_side_effect_features = [7983, 16186, 14315,  4342, 10051,  6958,  5749,  4451,  5001, 15755,  2222,  4654,  9280,  1746,  8412,  5861, 15848,  8946]
zero_side_effect_features_sorted_by_loss = [5861,  1746, 14315, 16186, 10051,  7983,  4342,  4654,  2222, 15755,  8412,  6958,  5749,  5001,  4451,  8946,  9280, 15848]

zero_side_effect_21_features = [ 5001, 11019,  3728,  7983,  9391,  4654, 14388,  5691,  4802, 1611,  7122,  4451, 14819, 15848, 14315, 12273, 15858,  4342, 12663, 12287]
zero_side_effect_21_features_sorted_by_loss = [ 9391, 12663,  7122, 11019,  3728,  7983, 14315,  4342,  4654, 15858, 12273, 14388,  1611,  5001,  4451,  5691, 14819, 15848, 12287,  4802]

unlearning_dataset = ['wmdp-bio']
side_effect_dataset_names =  ['high_school_us_history', 'college_computer_science', 'high_school_geography', 'human_aging', 'college_biology']
all_dataset_names = ['loss_added', 'wmdp-bio', 'high_school_us_history', 'college_computer_science', 'high_school_geography', 'human_aging', 'college_biology']

In [214]:
# Calculate metrics

main_ablate_params = {'multiplier': 25, 'intervention_method': 'clamp_feature_activation'}

sweep = {'features_to_ablate': features_to_test}
# sweep = {'features_to_ablate': [zero_side_effect_21_features_sorted_by_loss[:i+1] for i in range(len(zero_side_effect_21_features_sorted_by_loss))]}

dataset_names = all_dataset_names
# dataset_names = ['loss_added', 'human_aging']
# dataset_names = ['human_aging']
# dataset_names = ['loss_added']
# dataset_names = ['wmdp-bio']

n_batch_loss_added = 10

metrics_list = calculate_metrics_list(model,
                                      sae,
                                      main_ablate_params,
                                      sweep,
                                      dataset_names=dataset_names,
                                      include_baseline_metrics=False,
                                      n_batch_loss_added=n_batch_loss_added,
                                      activation_store=activation_store)


100%|██████████| 3/3 [00:00<00:00,  5.86it/s]


100%|██████████| 3/3 [00:00<00:00,  5.84it/s]


100%|██████████| 3/3 [00:00<00:00,  5.83it/s]


100%|██████████| 3/3 [00:00<00:00,  5.74it/s]


100%|██████████| 3/3 [00:00<00:00,  5.74it/s]


...

100%|██████████| 3/3 [00:00<00:00,  5.76it/s]


100%|██████████| 3/3 [00:00<00:00,  5.72it/s]


100%|██████████| 3/3 [00:00<00:00,  5.65it/s]


100%|██████████| 3/3 [00:00<00:00,  5.79it/s]


100%|██████████| 3/3 [00:00<00:00,  5.74it/s]

In [213]:
df = create_df_from_metrics(metrics_list)
df

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,0.000000,1.000000,1.0,1.0,1.000000,1.000,1.0,0.995535,0.991663,0.998721,0.997189,0.990612,0.996585
1,0.002897,0.988372,1.0,1.0,1.000000,1.000,1.0,0.988893,0.991663,0.998721,0.997189,0.990612,0.996374
2,-0.001240,0.860465,1.0,1.0,0.966667,0.875,1.0,0.972632,0.991663,0.998605,0.988468,0.985791,0.994973


In [206]:
# metrics_dict = convert_list_of_dicts_to_dict_of_lists(metrics_list)
df2 = create_df_from_metrics(metrics_list)
isort = np.argsort(df2['loss_added'].values[1:])
zero_side_effect_21_features_sorted_by_loss = np.array(zero_side_effect_21_features)[isort]

In [179]:
df = create_df_from_metrics(metrics_list)
df

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,NaN,1.000000,1.000000,1.000,1.000000,1.00000,1.000000,0.995535,0.991663,0.998721,0.997189,0.990612,0.996585
1,NaN,0.843023,0.666667,1.000,0.733333,0.68750,0.800000,0.875735,0.734524,0.939203,0.859293,0.871113,0.901451
2,NaN,0.779070,1.000000,1.000,1.000000,0.90625,0.866667,0.969053,0.991663,0.998721,0.997189,0.976653,0.961884
3,NaN,0.930233,1.000000,1.000,1.000000,1.00000,0.933333,0.988745,0.984314,0.998709,0.997189,0.990526,0.995830
4,NaN,0.988372,1.000000,1.000,1.000000,1.00000,1.000000,0.988893,0.991663,0.998721,0.997189,0.990612,0.996374
5,NaN,0.860465,1.000000,1.000,0.966667,0.87500,1.000000,0.972632,0.991663,0.998605,0.988468,0.985791,0.994973
6,NaN,0.953488,1.000000,1.000,1.000000,1.00000,1.000000,0.987520,0.991663,0.998721,0.997189,0.990602,0.996505
7,NaN,0.761628,0.925926,1.000,0.933333,0.93750,0.933333,0.962058,0.956138,0.998820,0.985955,0.990514,0.996789
8,NaN,1.000000,0.962963,1.000,1.000000,1.00000,1.000000,0.994656,0.995874,0.998838,0.997195,0.990065,0.996646
9,NaN,0.877907,1.000000,1.000,1.000000,1.00000,0.866667,0.978115,0.991664,0.998721,0.997181,0.990416,0.990677


In [192]:
keep_inds = df.query(("high_school_us_history == 1 & college_computer_science == 1 & high_school_geography == 1 & human_aging == 1")).index.values[1:]
keep_inds
zero_side_effect_21_features = np.array(features_to_test)[keep_inds]

In [193]:
zero_side_effect_21_features

array([ 5001, 11019,  3728,  7983,  9391,  4654, 14388,  5691,  4802,
        1611,  7122,  4451, 14819, 15848, 14315, 12273, 15858,  4342,
       12663, 12287])

In [46]:
# metrics_list[0]
metric_scores2 = np.array([[x[dataset_name]['mean_correct'] for dataset_name in dataset_names] for x in metrics_list])

df2 = pd.DataFrame(metric_scores2, columns=dataset_names)
df2 #df2.query(("high_school_us_history == 1 & college_computer_science == 1 & high_school_geography == 1 & human_aging == 1 & college_biology == 1"))

,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology
0,1.0,1.0,1.0,1.0,1.0,1.0


In [52]:
metrics_list[0]['wmdp-bio']['mean_predicted_probs']

0.9955348372459412

In [19]:
metric_scores2

array([[0.76162791, 1.        , 1.        , 1.        , 1.        ,
        0.86666673]])

In [13]:
isorted = np.argsort(([x['loss_added'] for x in metrics_list]))
zero_side_effect_features_sorted_by_loss = np.array(zero_side_effect_features)[isorted]
zero_side_effect_features_sorted_by_loss

array([ 5861,  1746, 14315, 16186, 10051,  7983,  4342,  4654,  2222,
       15755,  8412,  6958,  5749,  5001,  4451,  8946,  9280, 15848])

In [7]:
# # metrics_list[0]
# metric_scores2 = np.array([[x[dataset_name]['mean_correct'] for dataset_name in dataset_names] for x in metrics_list])

# df2 = pd.DataFrame(metric_scores2, columns=dataset_names)
# df2.query(("high_school_us_history == 1 & college_computer_science == 1 & high_school_geography == 1 & human_aging == 1 & college_biology == 1"))

UndefinedVariableError: name 'high_school_us_history' is not defined

In [77]:
# metrics_list[0]
metric_scores2 = np.array([[x[dataset_name]['mean_correct'] for dataset_name in dataset_names] for x in metrics_list])

df2 = pd.DataFrame(metric_scores2, columns=dataset_names)
df2.query(("high_school_us_history == 1 & college_computer_science == 1 & high_school_geography == 1 & human_aging == 1 & college_biology == 1"))

,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology
1,0.994186,1.0,1.0,1.0,1.0,1.0
2,0.976744,1.0,1.0,1.0,1.0,1.0
6,1.000000,1.0,1.0,1.0,1.0,1.0
7,0.930233,1.0,1.0,1.0,1.0,1.0
11,0.994186,1.0,1.0,1.0,1.0,1.0
15,0.994186,1.0,1.0,1.0,1.0,1.0
17,1.000000,1.0,1.0,1.0,1.0,1.0
19,0.994186,1.0,1.0,1.0,1.0,1.0
20,1.000000,1.0,1.0,1.0,1.0,1.0
21,1.000000,1.0,1.0,1.0,1.0,1.0


In [78]:
ikeep2 = [1, 2, 6, 7,11, 15, 17, 19, 20, 21]
features2 = np.array(possibly_bad_features)[ikeep2]
features2

array([ 5001, 15755,  2222,  4654,  9280,  1746,  8412,  5861, 15848,
        8946])

In [66]:
[x['mean_correct'] for x in metrics_dict['wmdp-bio']]

[0.9883720874786377, 0.9941860437393188, 0.9476743936538696]

In [67]:
metrics_dict = convert_list_of_dicts_to_dict_of_lists(metrics_list)


In [68]:
# metrics_list[0]
metric_scores = np.array([[x[dataset_name]['mean_correct'] for dataset_name in dataset_names] for x in metrics_list])

df = pd.DataFrame(metric_scores, columns=dataset_names)

In [72]:
df.query(("high_school_us_history == 1 & college_computer_science == 1 & high_school_geography == 1 & human_aging == 1 & college_biology == 1"))

,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology
0,0.988372,1.0,1.0,1.0,1.0,1.0
1,0.994186,1.0,1.0,1.0,1.0,1.0
3,0.988372,1.0,1.0,1.0,1.0,1.0
4,0.994186,1.0,1.0,1.0,1.0,1.0
5,0.994186,1.0,1.0,1.0,1.0,1.0
7,0.912791,1.0,1.0,1.0,1.0,1.0
13,0.994186,1.0,1.0,1.0,1.0,1.0
16,0.994186,1.0,1.0,1.0,1.0,1.0


In [80]:
ikeep = [0, 1, 3, 4, 5, 7, 13, 16]

In [81]:
features1 = np.array(filtered_features_sorted_by_loss)[ikeep]

In [82]:
zero_side_effect_features = np.concatenate((features1, features2))

In [83]:
zero_side_effect_features

array([ 7983, 16186, 14315,  4342, 10051,  6958,  5749,  4451,  5001,
       15755,  2222,  4654,  9280,  1746,  8412,  5861, 15848,  8946])

In [44]:
from unlearning.metrics import calculate_MCQ_metrics

In [49]:
geography_metrics = calculate_MCQ_metrics(
    model, 
    dataset_name='high_school_geography',
    # target_metric='correct',
    question_subset=None, 
    question_subset_file='../data/mmlu_high_school_geography_gemma-2b-it_correct_not_correct_wo_question_prompt.csv', 
    permutations=[[0, 1, 2, 3]], 
    verbose=True, 
    without_question=False,
)

100%|██████████| 5/5 [00:00<00:00,  6.66it/s]


In [51]:
from unlearning.metrics import save_target_question_ids

In [52]:
save_target_question_ids(model, 'high_school_geography')

100%|██████████| 792/792 [01:19<00:00,  9.96it/s]

Found correct questions:  34
Found correct questions but not correct without question prompt:  32


In [53]:
a = np.genfromtxt("../data/mmlu_high_school_geography_gemma-2b-it_correct.csv")
len(a)

30

In [54]:
a = np.genfromtxt("../data/mmlu_high_school_geography_gemma-2b-it_correct_not_correct_wo_question_prompt.csv")
len(a)

29

In [50]:
geography_metrics

{'mean_correct': 0.9655172228813171,
 'total_correct': 28,
 'is_correct': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32),
 'output_probs': array([[5.8606020e-06, 7.3286024e-08, 9.1389296e-08, 9.9019510e-01],
        [4.6751740e-07, 9.9216992e-01, 3.7255277e-06, 3.1369137e-07],
        [9.9833399e-01, 2.7094545e-08, 4.4471209e-09, 8.5933427e-10],
        [9.9753451e-01, 6.7145137e-08, 6.2250187e-08, 3.9717882e-08],
        [1.4488690e-03, 9.9394351e-01, 8.1123408e-06, 4.4798467e-06],
        [1.3650622e-04, 1.1737231e-06, 4.5552977e-05, 9.9140912e-01],
        [6.7609551e-05, 1.1535225e-03, 9.8921680e-01, 1.6691655e-06],
        [7.2710588e-07, 2.2885798e-09, 5.9943202e-09, 9.9846172e-01],
        [1.1759156e-03, 4.4876210e-06, 9.9412763e-01, 1.1222660e-06],
        [1.1058750e-05, 3.4316156e-06, 9.9368393e-01, 3.2170198e-07],
        [7.0274236e-06, 6.5973200e-07, 9.8619521e-01, 1.865099

In [16]:
len(metrics_list)

1

In [15]:
dataset_names

['wmdp-bio',
 'high_school_us_history',
 'college_computer_science',
 'high_school_geography',
 'human_aging',
 'college_biology']